In [7]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")
from src.utils import *
from src.graph2vec import ExtendedGraph2Vec, Ensemble_G2V
from sklearn.model_selection import LeaveOneOut
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def which_to_generate(num: int):
    path = (
        "../datasets/github_stargazers/"
        if num == 0
        else "../datasets/regular_graphs/"
        if num == 1
        else "../datasets/partition_graphs/"
    )
    graphs, labels = (
        read_stargazers_dataset("../datasets/github_stargazers")
        if num == 0
        else load_artificial(path)
    )
    order_dict = np.load(f"{path}orderings.npy", allow_pickle=True)[()]
    return graphs, labels, order_dict, path

In [9]:
DATASET_NUMBER = 1

In [10]:
# cv_fold = 64

graphs, labels, order_dict, path_to_save = which_to_generate(DATASET_NUMBER)

cv_fold = 10
single_emb_size = 128 if DATASET_NUMBER == 0 else 2 if DATASET_NUMBER == 1 else 15


tested_range = np.linspace(0, 1, 5)

WEIGHTING_FUNCTIONS = (
    {f"w_mean_{a:.2f}": ("w_mean", [a]) for a in tested_range}
    | {f"concat": ("concatenate", [])}
    | {f"p_proj_{a}": ("partial_projection", [0, a]) for a in (True, False)}
    | {f"w_proj_{a:.2f}": ("w_projection", [a]) for a in tested_range}
)

ens_model = Ensemble_G2V(
    ExtendedGraph2Vec(dimensions=single_emb_size),
    ExtendedGraph2Vec(use_pv_dm=True, dimensions=single_emb_size),
)

  0%|          | 0/475 [00:00<?, ?it/s]

In [11]:
res = []
for measure, order in tqdm(order_dict.items()):
    for name, (func, args) in tqdm(WEIGHTING_FUNCTIONS.items(), leave=False):
        ens_model.set_weighting_function(func, *args)
        res.append(
            cross_validate_graphs(
                graphs=graphs,
                ordering=order,
                labels=labels,
                n_splits=cv_fold,
                embedder=ens_model,
                cls=LogisticRegression(max_iter=1000),
                method=f"{measure}_{name}",
            )
        )

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
pd.concat(res, ignore_index=True).to_csv(f"{path_to_save}results2.csv")
pd.read_csv(f"{path_to_save}results2.csv", index_col=0).sample(15)

,true_label,graph_id,method,binary_prediction,fold,probability_prediction_0,probability_prediction_1,probability_prediction_2
70,1,70,pagerank_w_mean_0.00,1,1,0.210363,0.431639,0.357998
7982,1,382,closeness_centrality_w_mean_0.75,1,8,0.006056,0.882360,0.111584
9739,1,239,closeness_centrality_p_proj_False,1,5,0.224464,0.711024,0.064512
12787,2,437,current_flow_closeness_centrality_w_mean_0.00,1,8,0.153023,0.443632,0.403345
12269,1,394,closeness_centrality_w_proj_1.00,1,8,0.001986,0.996500,0.001514
13497,2,197,current_flow_closeness_centrality_w_mean_0.50,2,4,0.215511,0.139345,0.645144
16358,1,208,current_flow_closeness_centrality_w_proj_0.00,1,4,0.157882,0.563547,0.278571
9280,0,255,closeness_centrality_p_proj_True,0,4,0.964122,0.035042,0.000837
10921,0,471,closeness_centrality_w_proj_0.25,0,9,0.846847,0.000844,0.152309
14798,0,73,current_flow_closeness_centrality_concat,0,1,0.769270,0.167901,0.062829
